In [30]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
import unittest, time, re
import requests
from bs4 import BeautifulSoup
import string

def mkfile(soup,i):
    f = open('judicial_data/%s,%s.txt' %(i.decode('utf-8') , soup.findAll('td')[6].text), 'w')
    for i in range(5,11):
        f.write(soup.findAll('td')[i].text.encode('utf-8'))
        if i % 2 == 0:
            f.write('\n')
    f.write(soup.findAll('td')[13].text.encode('utf-8'))
list = ['臺灣基隆']
#'臺灣臺東','臺灣屏東','臺灣花蓮','福建金門'
#,'臺灣澎湖','福建連江'
list2 = ['31','30','30','31']

for i in list:
    for j in range(97,105):
        #89-105
        month = 1
        for k in list2:
            driver = webdriver.Firefox()
            #chromedriver = "C:\chromedriver_win32\chromedriver.exe"
            #driver = webdriver.Chrome(chromedriver)
            driver.implicitly_wait(30)
            driver.get("http://jirs.judicial.gov.tw//FJUD/FJUDQRY01_1.aspx")
            Select(driver.find_element_by_name("v_court")).select_by_visible_text("%s地方法院" % i)
            driver.find_elements_by_name('v_sys')[1].click() 
            driver.find_element_by_id("jt").clear()
            driver.find_element_by_id("jt").send_keys(u"離婚")
            driver.find_element_by_name("dy1").clear()
            driver.find_element_by_name("dy1").send_keys(str(j))
            driver.find_element_by_name("dm1").clear()
            driver.find_element_by_name("dm1").send_keys(str(month))
            driver.find_element_by_name("dd1").clear()
            driver.find_element_by_name("dd1").send_keys("1")
            driver.find_element_by_name("dy2").clear()
            driver.find_element_by_name("dy2").send_keys(str(j))
            driver.find_element_by_name("dm2").clear()
            driver.find_element_by_name("dm2").send_keys(str(month+2))
            driver.find_element_by_name("dd2").clear()
            driver.find_element_by_name("dd2").send_keys(k)            
            month += 3
            time.sleep(3)
            driver.find_element_by_name("Button").click()
            time.sleep(3)

            soup = BeautifulSoup(driver.page_source)
            alinks = soup.findAll('td')
            a = ''.join(alinks[4].text.split())
            m = re.match('.*?(\d+).*?(\d+).*?(\d+).*?', a)
            print m.group(1)

            driver.find_elements_by_xpath(u"(//a)")[2].click()
            time.sleep(3)
            soup = BeautifulSoup(driver.page_source)
            if soup.findAll('a')[1].text.encode('utf-8') == '第一筆':
                driver.find_element_by_link_text(u"第一筆").click()
                time.sleep(3)

            for num in range(1,int(m.group(1))):
                soup = BeautifulSoup(driver.page_source)
                mkfile(soup,i)
                driver.find_element_by_link_text(u"下一筆").click()
                time.sleep(3)
            driver.close()

KeyboardInterrupt: 